In [1]:
import numpy as np
import scipy.io as sio
import cvxpy as cvx
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
mat_contents = sio.loadmat('multiscenario.mat')

In [104]:
_mu = mat_contents['mu']
_V = mat_contents['V']
_lambda = mat_contents['lambda']

# convert into numpy arrays 
m = _mu.shape[0] # number of scenarios
d = _mu[0][0].shape[0] # dimension of the problem

mu = np.empty((m,d))
V = np.empty((m,d,d))
for i in range(m):
    mu[i,:] = np.ravel(_mu[i][0])
    V[i,:,:] = _V[i][0]

In [136]:
r = np.max(mu.ravel())/2
B = 10

x = cvx.Variable(d)
beta = cvx.Variable(1)


objective = cvx.Minimize(beta)
constraints = []

# volatility constriants
for k in range(m):
    constraints += [cvx.norm(V[k,:,:]*x) <= beta];

# mean constraints
for k in range(m):
    constraints += [mu[k,:].T*x >= r];

# portfolio constraint
constraints += [sum(x) == 1]

# absolute value constraints
constraints += [cvx.abs(x) <= B]

prob = cvx.Problem(objective,constraints)
prob.solve()



0.02641902038367077